교차검증 과 그리드 서치

In [1]:
#데이터 불러오기
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
#data target 분리
data = wine[['alcohol' , 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
#훈련, 테스트 , 검증 세트 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
#결정트리 모델 학습 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [9]:
#교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) #훈련시간, 검증시간, 검증점수 반환
print(scores)

{'fit_time': array([0.04753399, 0.01684141, 0.02369285, 0.02959561, 0.01221919]), 'score_time': array([0.00137496, 0.00117898, 0.01298904, 0.00119257, 0.00111628]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
#최종점수 출력
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 분할기 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
#폴드 수 변경
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [13]:
#그리드서치
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [14]:
#객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [15]:
#훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
#최적 하이퍼파라미터(매개변수)로 학습한 결과 출력
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [17]:
#최적의 매개변수 출력
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
#각 매개변수의 교차검증 점수 출력
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
#최소 샘플 수 찾기
params= {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
       'max_depth': range(5, 20, 1),
       'min_samples_split': range(2, 100, 10)}

In [22]:
#훈련
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
#최상의 매개변수 출력
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
#최상의 교차검증 점수 출력
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [26]:
#랜덤서치
from scipy.stats import uniform, randint
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1,25)}

In [29]:
from sklearn.model_selection import RandomizedSearchCV
gs  = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ef67be705b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ef67be73490>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ef67be722c0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ef67be71630>},
                   random_state=42)

In [30]:
#최적의 매개변수 출력
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [32]:
#최고의 교차검증 점수 출력
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884
